In [1]:
from dotenv import load_dotenv
import oracledb
import cx_Oracle
import os
import openai

load_dotenv()

True

# Generating SQL query equivalent to query in natural language through ChatGPT with context information

This approach consists of generating the SQL query equivalent to the natural language query in ChatGPT by adding a prompt that explains the Mondial schema.
A prompt was created explaining the Mondial schema that was passed to ChatGPT.
In order to verify that the sql generated by chatgptera is valid, it connected to the oracle database and ran the queries.

For simple queries it had good results, but for complex queries not. Plus we had to do a prompt to explain Mondial which was a lot of work and besides it would be bad to do a prompt for every database schema.

### Conectando ao Oracle Database
https://medium.com/oracledevs/using-the-development-branch-of-sqlalchemy-2-0-with-python-oracledb-d6e89090899c
https://stackoverflow.com/questions/14140902/using-oracle-service-names-with-sqlalchemy

In [3]:
username = str(os.getenv('DB_USER_NAME'))
passwd = str(os.getenv('DB_PASS'))
hostname = str(os.getenv('DB_HOST'))
port = str(os.getenv('DB_PORT'))
database = str(os.getenv('DB_NAME'))
sqldriver = str(os.getenv('SQL_DRIVER'))
servicename = str(os.getenv('SERVICE_NAME'))

# dsn = """(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST={hostname})
#     (PORT={port}))(CONNECT_DATA=(SERVER=DEDICATED)(SERVICE_NAME={servicename})))"""

print("ok")
dsnStr = cx_Oracle.makedsn(host = hostname, port = port, service_name = servicename).replace("SID", "SERVICE_NAME")    
conn = oracledb.connect(user=username, password=passwd,
                              dsn=dsnStr)


ok


#### Testando a conexão com uma consulta simples

In [4]:
def execute_sql(sql):
    if sql.endswith(";"):
        sql = sql[:-1]
    cursor = conn.cursor()
    cursor.execute(sql)
    rows = cursor.fetchall()
    for row in rows:
        print(row)
    cursor.close()

execute_sql("SELECT * FROM MONDIAL_TMDC;")



('http://www.semwebtech.org/mondial/AIRPORT', 'Airport', None)
('http://www.semwebtech.org/mondial/BORDERS', 'Borders', None)
('http://www.semwebtech.org/mondial/CITY', 'City', None)
('http://www.semwebtech.org/mondial/CITYOTHERNAME', 'Other Name City', None)
('http://www.semwebtech.org/mondial/CITYPOPS', 'Population City', None)
('http://www.semwebtech.org/mondial/CONTINENT', 'Continent', None)
('http://www.semwebtech.org/mondial/COUNTRY', 'Country', None)
('http://www.semwebtech.org/mondial/COUNTRYPOPS', 'Population Country', None)
('http://www.semwebtech.org/mondial/DESERT', 'Desert', None)
('http://www.semwebtech.org/mondial/ECONOMY', 'Economy', None)
('http://www.semwebtech.org/mondial/ENCOMPASSES', 'Encompasses', None)
('http://www.semwebtech.org/mondial/ETHNICGROUP', 'Ethnic Group', None)
('http://www.semwebtech.org/mondial/GEO_DESERT', 'Geo Desert', None)
('http://www.semwebtech.org/mondial/GEO_MOUNTAIN', 'Geo Mountain', None)
('http://www.semwebtech.org/mondial/ISLAND', 'Islan

### Criando a classe modelo do ChatGPT

In [5]:
openai.api_key = os.getenv("OPENAI_API_KEY")

class ChatModel:
    def __init__(self, prompt):
        self.messages = []
        self.messages.append({"role": "system", "content": prompt})
    
    def get_response(self, input):
        self.messages.append({"role": "user", "content": input})
        response =  openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages= self.messages,
            temperature=0.6,
            max_tokens=400
        )
        return response["choices"][0]["message"]["content"].strip()

### Carregando o prompt e criando um modelo

In [6]:
f = open("mondial_sql_prompt.txt", "r")
prompt = f.read()
f.close()
print(prompt)
chat = ChatModel(prompt)

You are a helpful assistant that generate sql query from a block of text and show the sql query. 

The Keywords must represent ENTITIES (TABLES) or PROPERTIES (ATTRIBUTES) or VALUES in my database

Below, we have the entities of my database:

Below is the list of ENTITIES or tables: 
AIRPORT
BORDERS
CITY
CONTINENT
COUNTRY
DESERT
ENCOMPASSES
ETHNICGROUP
ISLAND
LAKE
LANGUAGE
MOUNTAIN
ORGANIZATION
POLITICS
PROVINCE
RELIGION
RIVER
SEA
ECONOMY

When you generate the sql, you must place 'MONDIAL_' prefix in my tables. EX: MONDIAL_AIRPORT, MONDIAL_BORDERS, etc.

And below, we have my entities followed by their unique PROPERTIES in parentheses in my database:

AIRPORT (ELEVATION, GMTOFFSET, IATACODE, LATITUDE, LONGITUDE, NAME).
BORDERS (BORDERS ).
CITY (ELEVATION, LATITUDE, LONGITUDE, NAME, POPULATION).
CONTINENT (AREA, NAME).
COUNTRY (AREA, CAPITAL, CODE, NAME, POPULATION, PROVINCE).
DESERT (AREA, NAME, COORDINATES).
ENCOMPASSES (PERCENTAGE)
ETHNICGROUP (NAME, PERCENTAGE)
ISLAND (AREA, COORDI

### Executando Testes

##### SQL Query 

In [12]:
response = chat.get_response("What are the provinces with an area greater than 1000?")
print(response)
execute_sql(response)

SELECT META_REPCOL
FROM MONDIAL_PROVINCE
WHERE AREA > 1000
('Primorskiy',)
('Khabarovskiy',)
('Amurskaya',)
('Kamchatka',)
('Magadanskaya',)
('Sakhalin',)
('Khanty Mansi ao',)
('Yamalo Nenets ao',)
('Antwerpen',)
('Oost-Vlaanderen',)
('Hainaut',)
('LiÃ¨ge',)
('Limburg',)
('Luxembourg',)
('Namur',)
('West-Vlaanderen',)
('Brabant Wallon',)
('Vlaams-Brabant',)
('Luxembourg',)
('Groningen',)
('Friesland',)
('Drenthe',)
('Overijssel',)
('Flevoland',)
('Gelderland',)
('Utrecht',)
('Noord-Holland',)
('Zuid-Holland',)
('Zeeland',)
('Noord-Brabant',)
('Limburg',)
('Slovenia',)
('Croatia',)
('Federacija Bosne i Hercegovine',)
('Republika Srpska',)
('Bulgaria',)
('Alba',)
('Arad',)
('ArgeÅŸ',)
('BacÄƒu',)
('Bihor',)
('BistriÅ£a-NÄƒsÄƒud',)
('BotoÅŸani',)
('BrÄƒila',)
('BraÅŸov',)
('BuzÄƒu',)
('CÄƒlÄƒraÅŸi',)
('CaraÅŸ-Severin',)
('Cluj',)
('ConstanÅ£a',)
('Covasna',)
('DÃ¢mboviÅ£a',)
('Dolj',)
('GalaÅ£i',)
('Giurgiu',)
('Gorj',)
('Harghita',)
('Hunedoara',)
('IalomiÅ£a',)
('IaÅŸi',)
('MaramureÅŸ',

In [13]:
response = chat.get_response("What are the provinces with an area more than 1000?")
print(response)
execute_sql(response)

SELECT MONDIAL_PROVINCE.META_REPCOL
FROM MONDIAL_PROVINCE
WHERE MONDIAL_PROVINCE.AREA > 1000
('Primorskiy',)
('Khabarovskiy',)
('Amurskaya',)
('Kamchatka',)
('Magadanskaya',)
('Sakhalin',)
('Khanty Mansi ao',)
('Yamalo Nenets ao',)
('Antwerpen',)
('Oost-Vlaanderen',)
('Hainaut',)
('LiÃ¨ge',)
('Limburg',)
('Luxembourg',)
('Namur',)
('West-Vlaanderen',)
('Brabant Wallon',)
('Vlaams-Brabant',)
('Luxembourg',)
('Groningen',)
('Friesland',)
('Drenthe',)
('Overijssel',)
('Flevoland',)
('Gelderland',)
('Utrecht',)
('Noord-Holland',)
('Zuid-Holland',)
('Zeeland',)
('Noord-Brabant',)
('Limburg',)
('Slovenia',)
('Croatia',)
('Federacija Bosne i Hercegovine',)
('Republika Srpska',)
('Bulgaria',)
('Alba',)
('Arad',)
('ArgeÅŸ',)
('BacÄƒu',)
('Bihor',)
('BistriÅ£a-NÄƒsÄƒud',)
('BotoÅŸani',)
('BrÄƒila',)
('BraÅŸov',)
('BuzÄƒu',)
('CÄƒlÄƒraÅŸi',)
('CaraÅŸ-Severin',)
('Cluj',)
('ConstanÅ£a',)
('Covasna',)
('DÃ¢mboviÅ£a',)
('Dolj',)
('GalaÅ£i',)
('Giurgiu',)
('Gorj',)
('Harghita',)
('Hunedoara',)
('Ialo

In [15]:
response = chat.get_response("Tell me about cities of Cuba")
print(response)
execute_sql(response)

SELECT MONDIAL_CITY.META_REPCOL 
FROM MONDIAL_COUNTRY 
INNER JOIN MONDIAL_CITY ON MONDIAL_COUNTRY.CODE = MONDIAL_CITY.COUNTRY 
WHERE MONDIAL_COUNTRY.META_REPCOL LIKE '%Cuba%'
('CamagÃ¼ey',)
('Ciego de Ã¿vila',)
('Cienfuegos',)
('La Habana',)
('Bayamo',)
('Manzanillo',)
('GuantÃ¡namo',)
('HolguÃ\xadn',)
('Victoria de las Tunas',)
('Matanzas',)
('Pinar del RÃ\xado',)
('Sancti SpÃ\xadritus',)
('Santiago de Cuba',)
('Santa Clara',)
('Nueva Gerona',)
('Artemisa',)
('San JosÃ© de las Lajas',)


In [17]:
response = chat.get_response("city of Cuba")
print(response)
execute_sql(response)

SELECT * FROM MONDIAL_CITY WHERE COUNTRY = 'CUB'


In [18]:
response = chat.get_response("Say me about the total of city of cuba?")
print(response)
execute_sql(response)

SELECT COUNT(MONDIAL_CITY.META_REPCOL) 
    FROM MONDIAL_COUNTRY INNER JOIN MONDIAL_CITY ON MONDIAL_COUNTRY.CODE = MONDIAL_CITY.COUNTRY 
    WHERE MONDIAL_COUNTRY.META_REPCOL LIKE '%Cuba%'
(17,)


In [19]:
response = chat.get_response("Say me about the total of Cities of cuba?")
print(response)
execute_sql(response)

SELECT COUNT(MONDIAL_CITY.META_REPCOL) 
    FROM MONDIAL_COUNTRY INNER JOIN MONDIAL_CITY ON MONDIAL_COUNTRY.CODE = MONDIAL_CITY.COUNTRY 
    WHERE MONDIAL_COUNTRY.META_REPCOL LIKE '%Cuba%'
(17,)


In [20]:
response = chat.get_response("Say me about the total of cities of cuba?")
print(response)
execute_sql(response)

SELECT COUNT(*) 
FROM MONDIAL_CITY INNER JOIN MONDIAL_COUNTRY ON MONDIAL_CITY.COUNTRY = MONDIAL_COUNTRY.CODE 
WHERE MONDIAL_COUNTRY.META_REPCOL LIKE '%Cuba%'
(17,)


In [21]:
response = chat.get_response("Tell me the largest area.")
print(response)
execute_sql(response)

SELECT MONDIAL_COUNTRY.META_REPCOL, MONDIAL_COUNTRY.AREA
FROM MONDIAL_COUNTRY
ORDER BY MONDIAL_COUNTRY.AREA DESC
OFFSET 0 ROWS FETCH NEXT 1 ROWS ONLY
('Russia', 17075200)


SELECT MAX(MONDIAL_AREA.AREA) 
    FROM MONDIAL_AREA

SELECT MAX(MONDIAL_CONTINENT.AREA) 
    FROM MONDIAL_CONTINENT 
    
SELECT MAX(MONDIAL_COUNTRY.AREA) 
    FROM MONDIAL_COUNTRY
(17075200,)

In [22]:
response = chat.get_response("Areas different from 100")
print(response)
execute_sql(response)

SELECT MONDIAL_COUNTRY.META_REPCOL, MONDIAL_COUNTRY.AREA
FROM MONDIAL_COUNTRY
WHERE MONDIAL_COUNTRY.AREA <> 100;
('Lithuania', 65200)
('Belarus', 207600)
('Bosnia and Herzegovina', 51233)
('Ireland', 70280)
('Norway', 324220)
('Sweden', 449964)
('Lebanon', 10400)
('Syria', 185180)
('United Arab Emirates', 75581)
('Turkmenistan', 488100)
('Kyrgyzstan', 198500)
('Laos', 236800)
('Indonesia', 1919440)
('Papua New Guinea', 461690)
('American Samoa', 199)
('Mali', 1240000)
('Senegal', 196190)
('Ghana', 238540)
('Sudan', 1840687)
('Congo, Dem.Rep.', 2345410)
('Saint Helena', 391)
('Honduras', 112090)
('Turks and Caicos Islands', 430)
('Cuba', 110860)
('Curacao', 444)
('Aruba', 193)
('Andorra', 450)
('France', 547030)
('Estonia', 45100)
('Ukraine', 603700)
('Croatia', 56538)
('Malta', 316)
('Isle of Man', 588)
('Gaza Strip', 365)
('Qatar', 11000)
('Uzbekistan', 447400)
('Pakistan', 803940)
('India', 3287590)
('Bangladesh', 144000)
('Myanmar', 678500)
('Hong Kong', 1092)
('Philippines', 299764

In [23]:
response = chat.get_response("Shows me cities with longitude between 0 and 6")
print(response)
execute_sql(response)

SELECT MONDIAL_CITY.META_REPCOL 
    FROM MONDIAL_CITY 
    WHERE MONDIAL_CITY.LONGITUDE BETWEEN 0 AND 6
('Mons',)
('LiÃ¨ge',)
('Hasselt',)
('Arlon',)
('Namur',)
('Brugge',)
('Ostende',)
('Wavre',)
('Leuven',)
('Leeuwarden',)
('Lelystad',)
('Almere',)
('Arnhem',)
('Nijmegen',)
('Apeldoorn',)
('Utrecht',)
('Amersfoort',)
('Amsterdam',)
('Haarlem',)
('Zaanstad',)
('Hilversum',)
('Den Haag',)
('Rotterdam',)
('Dordrecht',)
('Leiden',)
('Middelburg',)
("'s-Hertogenbosch",)
('Eindhoven',)
('Tilburg',)
('Breda',)
('Maastricht',)
('Heerlen',)
('Sittard-Geleen',)
('Stavanger',)
('Bergen',)
('Bruxelles',)
('Schaerbeek',)
('Molenbeek-Saint-Jean',)
('Anderlecht',)
('Antwerpen',)
('Gent',)
('Charleroi',)
('Andorra la Vella',)
('Paris',)
('Boulogne-Billancourt',)
('Argenteuil',)
('Montreuil',)
('Saint-Denis',)
('Clermont-Ferrand',)
('Lyon',)
('Grenoble',)
('Saint-Ã‰tienne',)
('Villeurbanne',)
('Limoges',)
('Lille',)
('Amiens',)
('Calais',)
('Marseille',)
('Toulon',)
('Aix-en-Provence',)
('OrlÃ©ans',

In [24]:
response = chat.get_response("Shows me Airports with elevation more than 5000 ")
print(response)
execute_sql(response)

SELECT MONDIAL_AIRPORT.META_REPCOL 
FROM MONDIAL_AIRPORT
WHERE MONDIAL_AIRPORT.ELEVATION > 5000


In [25]:
response = chat.get_response("What language is spoken in Poland?")
print(response)
execute_sql(response)

SELECT MONDIAL_LANGUAGE.META_REPCOL 
    FROM MONDIAL_LANGUAGE INNER JOIN MONDIAL_COUNTRY ON MONDIAL_LANGUAGE.COUNTRY = MONDIAL_COUNTRY.CODE 
    WHERE MONDIAL_COUNTRY.META_REPCOL LIKE '%Poland%'
('German',)
('Polish',)


In [26]:
response = chat.get_response("Tell me about the religion that is practiced in Haiti")
print(response)
execute_sql(response)

SELECT MONDIAL_RELIGION.META_REPCOL 
    FROM MONDIAL_RELIGION INNER JOIN MONDIAL_COUNTRY ON MONDIAL_RELIGION.COUNTRY = MONDIAL_COUNTRY.CODE 
    WHERE MONDIAL_COUNTRY.META_REPCOL LIKE '%Haiti%'
('Catholic',)
('Protestant',)
('Voodoo',)


In [27]:
response = chat.get_response("How deep is Lake Kariba?")
print(response)
execute_sql(response)

SELECT DEPTH FROM MONDIAL_LAKE WHERE META_REPCOL = 'Lake Kariba'
(32,)


In [28]:
response = chat.get_response("What is the capital of Georgia?")
print(response)
execute_sql(response)

SELECT MONDIAL_COUNTRY.CAPITAL 
    FROM MONDIAL_COUNTRY 
    WHERE MONDIAL_COUNTRY.META_REPCOL LIKE '%Georgia%'
('Tbilisi',)


In [29]:
response = chat.get_response("What is the total of provinces of Netherlands?")
print(response)
execute_sql(response)

SELECT COUNT(MONDIAL_PROVINCE.META_REPCOL) 
    FROM MONDIAL_PROVINCE INNER JOIN MONDIAL_COUNTRY ON MONDIAL_PROVINCE.COUNTRY = MONDIAL_COUNTRY.CODE 
    WHERE MONDIAL_COUNTRY.META_REPCOL LIKE '%Netherlands%'
(12,)


In [30]:
response = chat.get_response("total provinces Netherland")
print(response)
execute_sql(response)

SELECT COUNT(MONDIAL_PROVINCE.META_REPCOL)
    FROM MONDIAL_PROVINCE
    WHERE MONDIAL_PROVINCE.META_REPCOL LIKE '%Netherlands%'
(0,)


In [31]:
response = chat.get_response("total Province Netherlands")
print(response)
execute_sql(response)

SELECT COUNT(META_REPCOL)
FROM MONDIAL_PROVINCE
WHERE META_REPCOL LIKE '%Netherlands%'
(0,)


In [32]:
response = chat.get_response("What percentage of religious people are hindu in thailand?")
print(response)
execute_sql(response)

SELECT MONDIAL_RELIGION.PERCENTAGE
FROM MONDIAL_RELIGION INNER JOIN MONDIAL_COUNTRY ON MONDIAL_RELIGION.COUNTRY = MONDIAL_COUNTRY.CODE
WHERE MONDIAL_RELIGION.META_REPCOL LIKE '%Hindu%' AND MONDIAL_COUNTRY.META_REPCOL LIKE '%Thailand%'
(0.1,)


In [33]:
response = chat.get_response("What are the provinces with an area greater than 1000 in Niger?")
print(response)
execute_sql(response)

SELECT MONDIAL_PROVINCE.META_REPCOL 
    FROM MONDIAL_PROVINCE 
    WHERE MONDIAL_PROVINCE.AREA > 1000 AND MONDIAL_PROVINCE.COUNTRY = 'NER'


In [40]:
response = chat.get_response("The Chatanga River belongs to which country?")
print(response)
execute_sql(response)

SELECT MONDIAL_COUNTRY.META_REPCOL 
    FROM MONDIAL_GEO_RIVER INNER JOIN MONDIAL_COUNTRY ON MONDIAL_GEO_RIVER.COUNTRY = MONDIAL_COUNTRY.CODE 
    WHERE MONDIAL_GEO_RIVER.RIVER LIKE '%Chatanga%'


DatabaseError: ORA-00942: table or view does not exist

In [41]:
response = chat.get_response("What province is the city of Glendale in?")
print(response)
execute_sql(response)

SELECT MONDIAL_PROVINCE.META_REPCOL 
    FROM MONDIAL_CITY INNER JOIN MONDIAL_PROVINCE ON MONDIAL_CITY.PROVINCE = MONDIAL_PROVINCE.META_REPCOL 
    WHERE MONDIAL_CITY.META_REPCOL LIKE '%Glendale%'
('Arizona',)
('California',)


In [44]:
response = chat.get_response("What is the area of the city Granada in Nicaragua?")
print(response)
execute_sql(response)

SELECT MONDIAL_CITY.AREA 
    FROM MONDIAL_CITY 
    WHERE MONDIAL_CITY.META_REPCOL LIKE '%Granada%' AND MONDIAL_CITY.COUNTRY = 'NIC'


DatabaseError: ORA-00904: "MONDIAL_CITY"."AREA": invalid identifier

In [51]:
response = chat.get_response("tell me about the Caribbean economic")
print(response)
execute_sql(response)

SELECT * 
FROM MONDIAL_ECONOMY 
WHERE MONDIAL_ECONOMY.META_REPCOL LIKE '%Caribbean%'


In [49]:
response = chat.get_response("Tell me about the country with the area largest.")
print(response)
execute_sql(response)

SELECT MONDIAL_COUNTRY.*
FROM MONDIAL_COUNTRY
ORDER BY MONDIAL_COUNTRY.AREA DESC
OFFSET 0 ROWS FETCH NEXT 1 ROWS ONLY
('Russia', 'R', 'Moskva', 17075200, 143666931, -0.03, 7.08, 'Russia')


In [50]:
response = chat.get_response("What is the host province of the International Energy Agency?")
print(response)
execute_sql(response)

SELECT MONDIAL_PROVINCE.META_REPCOL 
    FROM MONDIAL_PROVINCE INNER JOIN MONDIAL_ORGANIZATION ON MONDIAL_PROVINCE.META_REPCOL = MONDIAL_ORGANIZATION.PROVINCE 
    WHERE MONDIAL_ORGANIZATION.META_REPCOL LIKE '%International Energy Agency%'
('ÃŽle-de-France',)


In [47]:
response = chat.get_response("What is Mexico's independence day?")
print(response)
execute_sql(response)

SELECT MONDIAL_COUNTRY.INDEPENDENCE
FROM MONDIAL_COUNTRY
WHERE MONDIAL_COUNTRY.META_REPCOL LIKE '%Mexico%'


DatabaseError: ORA-00904: "MONDIAL_COUNTRY"."INDEPENDENCE": invalid identifier